# 1. Парсинг PDF и обновление JSON файлов

In [2]:
import json
import PyPDF2
from pathlib import Path

def parse_pdf_and_update_json(pdf_path, json_path):
    # Парсим PDF
    pdf_text = ""
    with open(pdf_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page in reader.pages:
            pdf_text += page.extract_text() + "\n"
    
    # Обновляем JSON
    with open(json_path, 'r+', encoding='utf-8') as json_file:
        data = json.load(json_file)
        data['Учебный_план'] = pdf_text
        json_file.seek(0)
        json.dump(data, json_file, ensure_ascii=False, indent=4)
        json_file.truncate()

# Обрабатываем оба файла
parse_pdf_and_update_json('AI Product.pdf', 'Управление ИИ-продуктами.json')
parse_pdf_and_update_json('Искусственный интеллект.pdf', 'Искусственный интеллект.json')

# 2. Создание векторной базы данных FAISS

In [2]:
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import os

def create_faiss_index():
    # Загружаем оба JSON файла
    docs = []
    for json_file in ['Искусственный интеллект.json', 'Управление ИИ-продуктами.json']:
        loader = JSONLoader(
            file_path=json_file,
            jq_schema='. | to_entries[] | {type: .key, content: .value}',
            text_content=False
        )
        docs.extend(loader.load())
    
    # Разбиваем текст на чанки
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(docs)
    
    # Создаем эмбеддинги и FAISS индекс
    embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")
    vectorstore = FAISS.from_documents(splits, embeddings)
    vectorstore.save_local("faiss_db")

create_faiss_index()

C:\Users\razum\AppData\Local\Temp\ipykernel_21352\168187333.py:26: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

c:\Users\razum\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\razum\.cache\huggingface\hub\models--cointegrated--LaBSE-en-ru. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

# 3. Чат-бот с RAG на LangChain

In [5]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate

class EduChatBot:
    def __init__(self):
        # Загружаем векторное хранилище с разрешением опасной десериализации
        self.embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")
        self.vectorstore = FAISS.load_local(
            "faiss_db", 
            self.embeddings,
            allow_dangerous_deserialization=True  # Добавляем этот параметр
        )
        
        # Инициализируем локальную LLM (Ollama)
        self.llm = Ollama(model="hermes3:latest", temperature=0.3)
        
        # Настраиваем промпт
        self.prompt_template = """Используй следующие фрагменты информации для ответа на вопрос.
        Если не знаешь ответа, просто скажи что не знаешь, не пытайся придумать ответ.
        
        Контекст: {context}
        
        Вопрос: {question}
        
        Полезный ответ:"""
        
        self.PROMPT = PromptTemplate(
            template=self.prompt_template,
            input_variables=["context", "question"]
        )
        
        # Создаем цепочку
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(),
            chain_type_kwargs={"prompt": self.PROMPT},
            return_source_documents=True
        )
    
    def ask(self, question):
        result = self.qa_chain({"query": question})
        return {
            "answer": result["result"],
            "sources": [doc.metadata["source"] for doc in result["source_documents"]]
        }

# Инициализация и использование бота
bot = EduChatBot()
response = bot.ask("Какие карьерные возможности у программы Искусственный интеллект?")
print("Ответ:", response["answer"])
print("Источники:", response["sources"])

C:\Users\razum\AppData\Local\Temp\ipykernel_21352\1729200064.py:18: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  self.llm = Ollama(model="hermes3:latest", temperature=0.3)
C:\Users\razum\AppData\Local\Temp\ipykernel_21352\1729200064.py:45: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.qa_chain({"query": question})


Ответ: В соответствии с предоставленными данными, программа в области искусственного интеллекта предлагает следующие карьерные возможности:

- От 150 до 400+ различных ролей в разных проектах на разных уровнях (стажер, младший, средний и старший специалист).

- Работу в таких областях как Middle: AI Product Manager - AI Project Manager - AI Product Data Analyst / AI Analyst - AI Product Lead.

Таким образом, программа предоставляет широкий спектр возможностей для развития карьеры в области искусственного интеллекта на разных позициях и уровнях.
Источники: ['C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Управление ИИ-продуктами.json', 'C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Управление ИИ-продуктами.json', 'C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Искусственный интеллект.json', 'C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Управление ИИ-продуктами.json']


In [6]:
response = bot.ask("Как можно поступить на программу Искусственный интеллект?")
print("Ответ:", response["answer"])
print("Источники:", response["sources"])

Ответ: Согласно предоставленным данным, для поступления на программу "Искусственный интеллект" требуется:

1. Математические и административные навыки и знания информационных технологий: 14 быдежтных, 0 целевой, 50 контрактных.

2. Информатическая и вычислительная техника и информация, связанная с системами: 51 быдежтное, 4 целевое, 55 контрактное место.

3. Информационные технологии и коммуникативность в сфере теннелогий и систем, а также системы связи: 80 быдежтных, 5 целевых, 25 контрактных мест.

Также указано, что на программу "Искусственный интеллект" принимаются AI-продуктты и технологии, поэтому для поступления важно иметь глубокое знание в сфере искусственного интеллекта.
Источники: ['C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Управление ИИ-продуктами.json', 'C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Управление ИИ-продуктами.json', 'C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Искусственный интеллект.json', 'C:\\Data\\Visual Studio\\Машинка практика\\

In [8]:
response = bot.ask("Принимаеться ли олимпиада «Я-профессионал»?")
print("Ответ:", response["answer"])
print("Источники:", response["sources"])

Ответ: Да, олимпиада "Я-профессионал" проводится. Она является реализацией проектов по всестороннему развитию технических и математических навыков и интересов школьников в разных возрастных группах.

Олимпиада состоит из нескольких этапов:
1) Математические и технические олимпиады для учащихся 7-9 классов
2) Информатика и программирование для учащихся 10-11 классов 
3) Информатика, программирование и телекоммуникации для учащихся 10-11 классов

В 2021 году на математических и технических олимпиадах приняло участие 14 тысяч, на информатики и программирования - 51 тысяча школьников. В целом по всем этапам было заявлено 80 тысяч участников, из которых 5 тысяч были девочками, а 25 тысяч - мальчиками.
Источники: ['C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Управление ИИ-продуктами.json', 'C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Искусственный интеллект.json', 'C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Управление ИИ-продуктами.json', 'C:\\Data\\Visual Studio\\Ма

In [10]:
response = bot.ask("Смогу ли я пользоваться льготами студентов очной формы обучения?")
print("Ответ:", response["answer"])
print("Источники:", response["sources"])

Ответ: Да, вы можете использовать льготы студентов очной формы обучения. Согласно предоставленным данным, в информатике и вячислителнии системах обучение получают 80 буджетов, 5 целевая, 25 контрактных мест. Таким образом, вы можете воспользоваться льготами студентов очной формы обучения для получения образования в указанных направлениях и системах.
Источники: ['C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Управление ИИ-продуктами.json', 'C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Искусственный интеллект.json', 'C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Управление ИИ-продуктами.json', 'C:\\Data\\Visual Studio\\Машинка практика\\Экзамен\\Искусственный интеллект.json']
